In [ ]:
#pip install indian-names
#pip install random-word
#pip install random

In [1]:
#imports
import pandas as pd
import indian_names
import random
from random_word import RandomWords
from datetime import datetime, timedelta
from faker import Faker
fake = Faker()

In [2]:
def generate_aadhar_number():
    return ' '.join(''.join(map(str,random.sample(range(10), 4))) for x in range(3))

In [3]:
def generate_passport_number():
    alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letter = random.choices(alphabets)[0]
    number =letter + str(random.randint(1,9)) + str(random.randint(0,9))\
                + ''.join(map(str,random.sample(range(10),4))) + str(random.randint(1,9)) 
    return number

In [4]:
def generate_phone_number():
    phone_number = ''
    
    #first number should be between 6-9
    phone_number += (str(random.randint(6,9)))
    
    for i in range(1, 10): 
        phone_number += (str(random.randint(0, 9))) 
        
    return phone_number

In [5]:
def generate_email(choices):
    number_choice = [''.join(map(str,random.sample(range(10),x))) for x in range(2,5)]
    domain_choice = ['gmail.com','yahoo.com','hotmail.com']
    return random.choices(choices,weights = [0.5,0.3,0.1,0.1])[0] + '.' + random.choices(choices,weights = [0.3,0.5,0.1,0.1])[0] + random.choice(number_choice) + '@' + random.choice(domain_choice)
    

In [6]:
class Passenger:
    def __init__(self):
        gender = random.choice(["male","female"])
        self.first_name = indian_names.get_first_name(gender)
        self.last_name = indian_names.get_last_name()
        self.age = random.randint(5, 80)
        if(self.age > 18):
            self.gender = random.choices([gender,"Others"],weights = [0.95,0.05])[0]
        else:
            self.gender = gender
        if(self.age < 15):
            self.contact_number = ""
            self.email_id = ""
            self.last_name = ""
        else: 
            self.contact_number = generate_phone_number()
            self.email_id = generate_email([self.first_name,self.last_name,RandomWords().get_random_word(),RandomWords().get_random_word()])
        self.aadhar_number = generate_aadhar_number()
        self.passport_number = random.choice([generate_passport_number(),""])

In [7]:
class Airplanes:
    def __init__(self,airplane_type,airplane_name):
        if(airplane_type == 'Boeing 777X'):
            self.fuel_capacity = 350410 #lbs
            self.seats = 384
            self.classes = {
                            "Economy" : { "seat_type" : "D"   ,"seats": 160 ,"window_seats": 30},
                            "Premium_Economy": { "seat_type" : "C" ,"seats": 104 , "window_seats": 25 } ,
                            "Business" : { "seat_type" : "B"   ,"seats": 70,"window_seats": 20} ,
                            "First_class": { "seat_type" : "A"   ,"seats": 50,"window_seats": 15}
                           }
            self.range = 16190 #km
            self.window_seats = 90
            self.base_price = 5000
        elif(airplane_type == 'Airbus A220'):
            self.fuel_capacity = 21805 
            self.seats = 160
            self.classes = {
                            "Economy" : { "seat_type" : "D"   ,"seats": 100 ,"window_seats": 30  },
                            "Premium_Economy": { "seat_type" : "C"   ,"seats": 60,"window_seats": 20}
                            }
            self.range = 6390 #km
            self.window_seats = 50
            self.base_price = 3000
        elif(airplane_type == 'Airbus A320'):
            self.fuel_capacity = 27200
            self.seats = 186
            self.range = 6150
            self.classes = {
                            "Economy" : { "seat_type" : "D"   ,"seats": 70 ,"window_seats": 20},
                            "Premium_Economy": { "seat_type" : "C" ,"seats": 50 , "window_seats": 20 } ,
                            "Business" : { "seat_type" : "B"   ,"seats": 40,"window_seats": 10} ,
                            "First_class": { "seat_type" : "A"   ,"seats": 26,"window_seats": 10}
                           }
            self.window_seats = 60
            self.base_price = 3500
            
        self.name = airplane_name
        self.airplane_id = airplane_name[:3] + '-' + airplane_type[:3]
        self.type = airplane_type
        self.current_fuel = 0
         

In [8]:
class Airport:
    def __init__(self,airport_name):
        self.name = airport_name
        self.code = airport_name[:3].upper()
        self.airplanes = set()
        

In [31]:
class Flight:
    def __init__(self,airplane,source_airport,destination_airport,flight_id):
        curr_airplane = airplane   #We will pick each plane and check whether it wants to go or no
                
        #current airplane data
        self.id = flight_id
        self.total_seats  = curr_airplane.seats  
        self.flight_code = curr_airplane.airplane_id
        self.classes = curr_airplane.classes
        self.base_price = curr_airplane.base_price
        self.departure_time = fake.date_time_between(start_date='-1d', end_date='+1d')
        self.arrival_time = self.departure_time + timedelta(hours=random.randint(1, 12))
        self.source = source_airport
        self.destination = destination_airport
        self.passengers = [] 
        self.status = None
        self.comments = None
       
        
    def get_flight_data(self):
        return {
            "id" : self.id
            ,"flight_code" : self.flight_code
            ,"source" : self.source
            ,"destination" : self.destination
            ,"departure_time" : self.departure_time
            ,"arrival_time" : self.arrival_time
            ,"passenger_data" : self.passengers
            ,"total_seats" : self.total_seats      
        }
        
    def fill_flight(self):
        base_price = self.base_price
        flight_classes = self.classes
        for flight_class in list(flight_classes.keys()):
            total_seats = flight_classes[flight_class]["seats"]
            window_seats = flight_classes[flight_class]["window_seats"]
            seat_type = flight_classes[flight_class]["seat_type"]
            while total_seats > 0:
                boarding = random.choices(["Yes","No"],weights = [0.9,0.1])[0]
                if(boarding):
                        cash = base_price
                        if(seat_type == 'A'):
                            cash += base_price*0.4
                        elif(seat_type == 'B'):
                            cash += base_price*0.3
                        elif(seat_type == 'C'):
                            cash += base_price*0.2
    
                        seat_number = seat_type +'-'+ str(total_seats)
                        
                        window = random.choices(["Yes","No"],weights = [0.4,0.6])[0]
                        if(window_seats > 0 and window):
                            seat_number = seat_number + '(W)'
                            cash += base_price * 0.1
                            window_seats -= 1
                        
                        passenger = Passenger()
                        
                        passenger_data = {
                            "first_name" : passenger.first_name
                            ,"last_name" : passenger.last_name
                            ,"age" : passenger.age
                            ,"gender" : passenger.gender
                            ,"contact_number" : passenger.contact_number
                            ,"email_id" : passenger.email_id
                            ,"aadhar_number" : passenger.aadhar_number
                            ,"passport_number" : passenger.passport_number
                            ,"seat": seat_number
                            ,"fare" : cash    
                        }
                            
                        self.passengers.append(passenger_data)                     
                            
                total_seats -= 1
                #print(f'Loading...{total_seats}')
        
        
    

In [45]:
class Zathura:
    
    #Constructor
    def __init__(self):
        
        #Initialization the harcoded value + data structures
        self.global_flight_id = 100001  
        self.aircraft_models = {
            'Boeing 777X': ['Garuda', 'Vimana', 'Vayu', 'Ananta'],
            'Airbus A220': ['Agni', 'Astra', 'Vajra'],
            'Airbus A320': ['Rudra', 'Marut']
        }
        self.airports = ["Mumbai","Delhi","Pune","Banglore"]
        self.airport_data = {}
        self.flights_data = []
        self.airplane_data = []
     
    
    #Setting airport data
    def set_airport_data(self):
        for airport in self.airports:
            self.airport_data[airport] = (Airport(airport)) 
            
    #Setting airplane_data
    def set_airplane_data(self):
        for model in self.aircraft_models:
                for model_name in self.aircraft_models[model]:
                        self.airplane_data.append(Airplanes(model,model_name))
           
    #Setting airplanes to a initial airport
    def set_inital_airplanes_to_airports(self):
        for airplane in self.airplane_data:
            self.airport_data["Pune"].airplanes.add(airplane)
    
    #Schedule flights
    def schedule_flights(self):
        
        #trackers are used to remove and add airplanes from source to destination airport
        destination_tracker = { airport_name : set() for airport_name in self.airports}
        source_tracker = { airport_name : set() for airport_name in self.airports}
        
        #Will go through all airports and schedule flight for each airplane present in the airport
        for airport in self.airports:
            
            source_airport = airport          
            source_airport_data = self.airport_data[source_airport] 
            source_airplanes = source_airport_data.airplanes
            
            #running for each airplane
            for airplane in source_airplanes:
                
                #finding destination
                destination_airport = source_airport
                while(destination_airport == source_airport):
                    destination_airport = random.choice(self.airports)
                
                #scheduling_flight
                flight = Flight(airplane,source_airport,destination_airport,self.global_flight_id)
                flight.fill_flight()
                self.flights_data.append(flight.get_flight_data())
                
                #managing trackers
                source_tracker[source_airport].add(airplane)
                destination_tracker[destination_airport].add(airplane) 
                
                self.global_flight_id += 1
         
        #fixing trackers for the next run
        for airport in self.airports:
            self.airport_data[airport].airplanes = self.airport_data[airport].airplanes | destination_tracker[airport]
            self.airport_data[airport].airplanes = self.airport_data[airport].airplanes - source_tracker[airport]
                
    def get_print(self):
        for airport in self.airports:
            curr_airport = self.airport_data[airport]
            print(f"{curr_airport.name}- ")
            for ap in curr_airport.airplanes:
                print(f" \t\t {ap.name}")
                
                
    def show_data(self):
        flight_schedules_df = pd.DataFrame(self.flights_data)
        display(flight_schedules_df)
        
        

In [46]:
zathura = Zathura()
zathura.set_airplane_data()
zathura.set_airport_data()
zathura.set_inital_airplanes_to_airports()

Mumbai- 
Delhi- 
Pune- 
 		 Agni
 		 Ananta
 		 Vayu
 		 Garuda
 		 Rudra
 		 Vajra
 		 Marut
 		 Vimana
 		 Astra
Banglore- 
Loading...99
Loading...98
Loading...97
Loading...96
Loading...95
Loading...94
Loading...93
Loading...92
Loading...91
Loading...90
Loading...89
Loading...88
Loading...87
Loading...86
Loading...85
Loading...84
Loading...83
Loading...82
Loading...81
Loading...80
Loading...79
Loading...78
Loading...77
Loading...76
Loading...75
Loading...74
Loading...73
Loading...72
Loading...71
Loading...70
Loading...69
Loading...68
Loading...67
Loading...66
Loading...65
Loading...64
Loading...63
Loading...62
Loading...61
Loading...60
Loading...59
Loading...58
Loading...57
Loading...56
Loading...55
Loading...54
Loading...53
Loading...52
Loading...51
Loading...50
Loading...49
Loading...48
Loading...47
Loading...46
Loading...45
Loading...44
Loading...43
Loading...42
Loading...41
Loading...40
Loading...39
Loading...38
Loading...37
Loading...36
Loading...35
Loading...34
Loading...33
Loa

Loading...87
Loading...86
Loading...85
Loading...84
Loading...83
Loading...82
Loading...81
Loading...80
Loading...79
Loading...78
Loading...77
Loading...76
Loading...75
Loading...74
Loading...73
Loading...72
Loading...71
Loading...70
Loading...69
Loading...68
Loading...67
Loading...66
Loading...65
Loading...64
Loading...63
Loading...62
Loading...61
Loading...60
Loading...59
Loading...58
Loading...57
Loading...56
Loading...55
Loading...54
Loading...53
Loading...52
Loading...51
Loading...50
Loading...49
Loading...48
Loading...47
Loading...46
Loading...45
Loading...44
Loading...43
Loading...42
Loading...41
Loading...40
Loading...39
Loading...38
Loading...37
Loading...36
Loading...35
Loading...34
Loading...33
Loading...32
Loading...31
Loading...30
Loading...29
Loading...28
Loading...27
Loading...26
Loading...25
Loading...24
Loading...23
Loading...22
Loading...21
Loading...20
Loading...19
Loading...18
Loading...17
Loading...16
Loading...15
Loading...14
Loading...13
Loading...12
Loading...11

Loading...15
Loading...14
Loading...13
Loading...12
Loading...11
Loading...10
Loading...9
Loading...8
Loading...7
Loading...6
Loading...5
Loading...4
Loading...3
Loading...2
Loading...1
Loading...0
Loading...49
Loading...48
Loading...47
Loading...46
Loading...45
Loading...44
Loading...43
Loading...42
Loading...41
Loading...40
Loading...39
Loading...38
Loading...37
Loading...36
Loading...35
Loading...34
Loading...33
Loading...32
Loading...31
Loading...30
Loading...29
Loading...28
Loading...27
Loading...26
Loading...25
Loading...24
Loading...23
Loading...22
Loading...21
Loading...20
Loading...19
Loading...18
Loading...17
Loading...16
Loading...15
Loading...14
Loading...13
Loading...12
Loading...11
Loading...10
Loading...9
Loading...8
Loading...7
Loading...6
Loading...5
Loading...4
Loading...3
Loading...2
Loading...1
Loading...0
Loading...69
Loading...68
Loading...67
Loading...66
Loading...65
Loading...64
Loading...63
Loading...62
Loading...61
Loading...60
Loading...59
Loading...58
Loadin

Loading...120
Loading...119
Loading...118
Loading...117
Loading...116
Loading...115
Loading...114
Loading...113
Loading...112
Loading...111
Loading...110
Loading...109
Loading...108
Loading...107
Loading...106
Loading...105
Loading...104
Loading...103
Loading...102
Loading...101
Loading...100
Loading...99
Loading...98
Loading...97
Loading...96
Loading...95
Loading...94
Loading...93
Loading...92
Loading...91
Loading...90
Loading...89
Loading...88
Loading...87
Loading...86
Loading...85
Loading...84
Loading...83
Loading...82
Loading...81
Loading...80
Loading...79
Loading...78
Loading...77
Loading...76
Loading...75
Loading...74
Loading...73
Loading...72
Loading...71
Loading...70
Loading...69
Loading...68
Loading...67
Loading...66
Loading...65
Loading...64
Loading...63
Loading...62
Loading...61
Loading...60
Loading...59
Loading...58
Loading...57
Loading...56
Loading...55
Loading...54
Loading...53
Loading...52
Loading...51
Loading...50
Loading...49
Loading...48
Loading...47
Loading...46
Load

In [ ]:
zathura.get_print()
zathura.schedule_flights()
zathura.get_print()